# Fits over Directory
Run analysis procedures over directory to get fits and plots. Probably have to do manual labour but lets see...

In [ ]:
#importing directories
from random import gauss
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import os
from astropy.io import ascii, fits
#from ellc import lc
from transit import occultnonlin, occultquad
from scipy.signal import find_peaks

Global Constants over entire code

In [ ]:
rv_entry=ascii.read('../../Catalogs/robovetter_label.dat')
rv_pl=np.array(rv_entry['tce_plnt_num'])
rv_label = np.array(rv_entry['label'])
rv_kepid=[('0000'+str(el)[:9])[-9:] for el in rv_entry['kepid']]
u1=np.array(rv_entry['u1'])
u2=np.array(rv_entry['u2'])
b=np.array(rv_entry['b'])
rpl=np.array(rv_entry['rpl_rstar'])
rorb=np.array(rv_entry['rorb_rstar'])



Fitting Functions: Gaussian, Lorentzian, Planet Transit

In [ ]:
def gausses(x, A1, m1, s1, A2, m2, s2):
    y = A1*np.exp(-(x-m1)**2/(2*s1**2)) + A2*np.exp(-(x-m2)**2/(2*s2**2))
    return(y)

def lorz(x,A1,x0,g, A2, x02, g2):
    y = A1 / (1*((x-x0)**2+(g/2)**2)) + A2 / (1*((x-x02)**2+(g2/2)**2))
    return(y)

def new_plar(ph,p,minus, plus,rorb):
    u1 = (plus + minus)/2
    u2 = (plus - minus)/2
    znp = np.abs(rorb*np.sin(ph*np.pi))
    a= occultquad(znp,p,[u1,u2])  
    return(a -1) 

def new_plar_free(ph,p,u1,plus,rorb):
    u2 = plus - u1
    znp = np.abs(rorb*np.sin(ph*np.pi))
    a= occultquad(znp,p,[u1,u2])  
    return(a -1) 

def new_plar_free2(ph,p,u1,u2,rorb, imp):
    znp = np.sqrt(np.abs(rorb*np.sin(ph*np.pi))**2+imp**2)
    a= occultquad(znp,p,[u1,u2])  
    return(a -1) 

# def new_plar_v2(ph,p,u1,u2,rorb, imp):
#     incl = np.arccos(imp/rorb)*180/np.pi
#     a= lc(ph,radius_1=1/rorb,radius_2=p/rorb,sbratio=0.0,incl=incl,ld_1='quad',ldc_1=[u1,u2],
#         shape_1='sphere', shape_2='sphere')
#     return(a -1) 

def smooth(arr,k=10):
    kernel = np.ones(k)/k
    v=np.convolve(arr, kernel, mode='same')
    return(v)


### Improved fitting
Not fitting models, but rather developing models from already fitted parameters in the catalog. Best way to filter bad stuff.
No rebin to 200 pixel local view ... using the full extent of the resolution.

In [ ]:

def sort_data(hdu, hduno):
    flux = []
    flux_white=[]
    phase = []
    model = []
    tp = hdu[hduno].header['TPERIOD']
    for ph, fl,wl, ml in zip(hdu[hduno].data['PHASE'],hdu[hduno].data['LC_DETREND'],hdu[hduno].data['LC_WHITE'],hdu[hduno].data['MODEL_WHITE']):
        if not np.isnan(fl):
            flux.append(fl)
            if(ph*2/tp>1): phase.append(ph*2/tp -2)
            else: phase.append(ph*2/tp)
            flux_white.append(wl)
            model.append(ml)
    dfunb = pd.DataFrame(list(zip(phase, flux, flux_white, model)),columns=['phase', 'flux','flux_white', 'model'])
    df=dfunb.sort_values('phase',axis=0,ascending=True)
    df=df[(df.phase>-0.75) & (df.phase<0.75)]

    bins=np.linspace(min(df['phase']),max(df['phase']),8000)
    groups = df.groupby(np.digitize(df['phase'], bins))
    df=groups.median()
    
    # print('size:',len(df)) 
    return(df, hdu[hduno].header['TPERIOD'], hdu[hduno].header['TDUR']/24)

def get_fit_residuals(df, kid, tp ,td, el):
    try: lc=new_plar_free2(np.array(df['phase']), rpl[kid],u1[kid], u2[kid], rorb[kid], b[kid])
    except: 
        print('weird error')
        return(0)
    res = np.array(df['flux'])-lc
    df=pd.DataFrame(list(zip(df['phase'], df['flux'], df['flux_white'], df['model'],lc, res)),
        columns=['phase', 'flux','flux_white', 'model','par','res'])
    
    locdf=df[(df.phase>-2*td/tp) & (df.phase<2*td/tp)]
    out = df[(df.phase<-2*td/tp) | (df.phase>2*td/tp)]
    
    noise = np.std(np.array(out['res']))
    # popt1, pcov1 = curve_fit(gausses, locdf['phase']*0.9, locdf['res'], p0=[max(res),min(locdf['phase'])/2,0.01,max(res),max(locdf['phase'])/2,0.01],
    #     bounds=([max(res)/2,min(locdf['phase'])*0.9,
    #     0.001,max(res)/2,0,0.001], [max(res), 0, np.inf,max(res), max(locdf['phase'])*0.9,np.inf]))
    res=np.array(locdf['res'])
    ph = np.array(locdf['phase'])
    peaks, _ = find_peaks(res, height=max(res)/2, distance=30)
    try: ins=max([res[x] for x in peaks if(ph[x]<0 and ph[x]>min(ph)/1.9)])
    except: ins=0
    # if(popt1[1]>noise and popt1[4]>noise and popt1[2]-popt1[5]< td/tp): print('bingo')

    #print(ph[peaks])
    #plotting everything:
    fig, ax=plt.subplots(3,1, figsize=(7,10))
    ax[0].plot(df['phase'],df['flux'], label='flux')
    ax[0].plot(df['phase'],lc, label='model')
    ax[0].set_ylabel('flux')

    ax[1].plot(locdf['phase'],locdf['flux'], label='flux')
    ax[1].plot(locdf['phase'],locdf['par'], label='model')
    
    
    plt.suptitle(el+':'+str(np.array(rv_label)[kid]))
    ax[0].set_title('rpl'+str(np.around(rpl[kid],2))+'u1'+str(np.around(u1[kid],2))+'u2'+str(np.around(u2[kid],2))+
        'rorb'+str(np.around(rorb[kid],2))+'b'+str(np.around(b[kid],2)))
    # ax[2].set_title('a1:'+str(np.around(popt1[0],5))+' m1: '+str(np.around(popt1[1],2))+' s1: '+str(np.around(popt1[2],2))+' a2: '
    #     +str(np.around(popt1[3],5))+ 'm2: '+str(np.around(popt1[4],2))+' s2: '+str(np.around(popt1[5],2)))
    ax[2].set_title('snr:'+str(np.around(ins/noise,4)))
    ax[2].plot(locdf['phase'],locdf['res'], label='residual')
    #ax[2].plot(locdf['phase'],gausses(locdf['phase'], *popt1), label='fit')
    
    ax[2].plot(ph[peaks], res[peaks], "o")
    ax[2].set_xlabel('phase')
    ax[2].set_ylabel('flux - model')
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

    #store data
    c1=np.array([(x<0 and x>min(ph)/1.9) for x in ph[peaks]]).sum()
    c2=np.array([(x>0 and x<max(ph)/1.9) for x in ph[peaks]]).sum()

       

    if(c1>0 and c2>0 and len(ph[peaks])-c1-c2<c1+c2 and ins/noise>3): 
        print('yay', c1, c2,len(ph[peaks])-c1-c2)
        store = pd.HDFStore('../../processed_directories/go_circles/new_data/'+el)
        store.put('data', df)
        store.get_storer('data').attrs.metadata = {'ploc':ph[peaks],'pval':res[peaks],'label':rv_label[kid],'rorb':rorb[kid],'rpl':rpl[kid]
            ,'u1':u1[kid],'u2':u2[kid],'b':b[kid],'tp':tp,'td':td, 'snr':ins/noise}
        store.close()
        plt.savefig('../../processed_directories/go_circles/new_fits/'+el+'.jpg')
    else: 
        print('nay', c1, c2, len(ph[peaks])-c1-c2)
        store = pd.HDFStore('../../processed_directories/go_circles/new_data_stash/'+el)
        store.put('data', df)
        store.get_storer('data').attrs.metadata = {'ploc':ph[peaks],'pval':res[peaks],'label':rv_label[kid],'rorb':rorb[kid],'rpl':rpl[kid]
            ,'u1':u1[kid],'u2':u2[kid],'b':b[kid],'tp':tp,'td':td, 'snr':ins/noise}
        store.close()
        plt.savefig('../../processed_directories/go_circles/new_fits_stash/'+el+'.jpg')

    # df['mfitres']=gausses(df['phase'], *popt1)

    plt.close()

def get_fit_residuals_crude(df, tp ,td, el):
    print(el)
    
    res = np.array(df['flux_white'])-np.array(df['model'])
    df=pd.DataFrame(list(zip(df['phase'], df['flux'], df['flux_white'], df['model'], res)),
        columns=['phase', 'flux','flux_white', 'model','res'])
    
    locdf=df[(df.phase>-2*td/tp) & (df.phase<2*td/tp)]
    out = df[(df.phase<-2*td/tp) | (df.phase>2*td/tp)]
    
    noise = np.std(np.array(out['res']))
    # popt1, pcov1 = curve_fit(gausses, locdf['phase']*0.9, locdf['res'], p0=[max(res),min(locdf['phase'])/2,0.01,max(res),max(locdf['phase'])/2,0.01],
    #     bounds=([max(res)/2,min(locdf['phase'])*0.9,
    #     0.001,max(res)/2,0,0.001], [max(res), 0, np.inf,max(res), max(locdf['phase'])*0.9,np.inf]))
    res=np.array(locdf['res'])
    ph = np.array(locdf['phase'])
    try: 
        peaks, _ = find_peaks(res, height=max(res)/2, distance=30)
        ins=max([res[x] for x in peaks if(ph[x]<0 and ph[x]>min(ph)/1.9)])
    except:
         ins=0
         return(0)
    # if(popt1[1]>noise and popt1[4]>noise and popt1[2]-popt1[5]< td/tp): print('bingo')

    #print(ph[peaks])
    #plotting everything:
    fig, ax=plt.subplots(3,1, figsize=(7,10))
    ax[0].plot(df['phase'],df['flux_white'], label='flux')
    ax[0].plot(df['phase'],df['model'], label='model')
    ax[0].set_ylabel('flux')

    ax[1].plot(locdf['phase'],locdf['flux_white'], label='flux')
    ax[1].plot(locdf['phase'],locdf['model'], label='model')
    
    
    plt.suptitle(el)
    ax[0].set_title('rpl'+str(np.around(rpl[kid],2))+'u1'+str(np.around(u1[kid],2))+'u2'+str(np.around(u2[kid],2))+
        'rorb'+str(np.around(rorb[kid],2))+'b'+str(np.around(b[kid],2)))
    # ax[2].set_title('a1:'+str(np.around(popt1[0],5))+' m1: '+str(np.around(popt1[1],2))+' s1: '+str(np.around(popt1[2],2))+' a2: '
    #     +str(np.around(popt1[3],5))+ 'm2: '+str(np.around(popt1[4],2))+' s2: '+str(np.around(popt1[5],2)))
    ax[2].set_title('snr:'+str(np.around(ins/noise,4)))
    ax[2].plot(locdf['phase'],locdf['res'], label='residual')
    #ax[2].plot(locdf['phase'],gausses(locdf['phase'], *popt1), label='fit')
    
    ax[2].plot(ph[peaks], res[peaks], "o")
    ax[2].set_xlabel('phase')
    ax[2].set_ylabel('flux - model')
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

    #store data
    c1=np.array([(x<0 and x>min(ph)/1.9) for x in ph[peaks]]).sum()
    c2=np.array([(x>0 and x<max(ph)/1.9) for x in ph[peaks]]).sum()

       

    if(c1>0 and c2>0 and len(ph[peaks])-c1-c2<c1+c2 and ins/noise>3): 
        print('yay', c1, c2,len(ph[peaks])-c1-c2)
        store = pd.HDFStore('../../processed_directories/go_circles/new_data_2/'+el)
        store.put('data', df)
        store.get_storer('data').attrs.metadata = {'ploc':ph[peaks],'pval':res[peaks],'label':rv_label[kid],'rorb':rorb[kid],'rpl':rpl[kid]
            ,'u1':u1[kid],'u2':u2[kid],'b':b[kid],'tp':tp,'td':td, 'snr':ins/noise}
        store.close()
        plt.savefig('../../processed_directories/go_circles/new_fits_2/'+el+'.jpg')
    else: 
        print('nay', c1, c2, len(ph[peaks])-c1-c2)
        store = pd.HDFStore('../../processed_directories/go_circles/new_data_stash_2/'+el)
        store.put('data', df)
        store.get_storer('data').attrs.metadata = {'ploc':ph[peaks],'pval':res[peaks],'tp':tp,'td':td, 'snr':ins/noise}
        store.close()
        plt.savefig('../../processed_directories/go_circles/new_fits_stash_2/'+el+'.jpg')

    # df['mfitres']=gausses(df['phase'], *popt1)

    plt.close()


In [ ]:
FILEPATH_FPS="E:\Masters_Project_Data\\alienworlds_fps\\"
entries = os.listdir(FILEPATH_FPS)

for el in entries[8000:]:
    hdu = fits.open(FILEPATH_FPS+el)
    print(el[4:13],"TCEs:", len(hdu)-2)
    kid=np.where(np.array(rv_kepid)==str(el[4:13]))[0]
    if(len(kid)==0): 
        print('not in catalog')
        continue

    for n in range(1,len(hdu)-1):
        kidf=[x for x in kid if(rv_pl[x]==n)] 
        if(len(kidf)==0): 
            print('not in catalog')
            continue
        kidf=kidf[0]
        df, tp, td = sort_data(hdu, n)
        print(rpl[kidf],u1[kidf], u2[kidf], rorb[kidf], b[kidf], tp, td)
        get_fit_residuals(df, kid, tp, td, el[4:13]+'_'+str(n))

        # df, tp, td = sort_data(hdu, n)
        # if(tp==0 or td==0):
        #     print('not fit') 
        #     continue
        # get_fit_residuals_crude(df, tp, td, el[4:13]+'_'+str(n))

    hdu.close()
        

    

In [ ]:
#plt.plot(hdu[1].data['PHASE']/tp, hdu[1].data['LC_WHITE'], marker='.', ls='None')
#plt.plot(df['phase'], df['flux'])
entries = os.listdir(FILEPATH_FPS)
hdu = fits.open(FILEPATH_FPS+entries[0])
m=entries[0]
kid=np.where(np.array(rv_kepid)==str(m[4:13]))[0]
print(kid, m)
df, tp, td = sort_data(hdu, 1)
plt.plot(df['phase'], df['flux'])
print(rpl[kid],u1[kid], u2[kid], rorb[kid], b[kid])
lc=new_plar_free2(df['phase'], rpl[kid],u1[kid], u2[kid], rorb[kid], b[kid])
plt.plot(df['phase'],lc)
plt.xlim(-0.05,0.05)
#plt.plot(hdu[1].data['P
# HASE']/tp, hdu[1].data['MODEL_WHITE'], marker='.', ls='None')

In [ ]:
from scipy.signal import find_peaks

fig, ax = plt.subplots(2,1)
df=df[(df.phase>-0.02) & (df.phase<0.02)]
lc=new_plar_free2(df['phase'], rpl[kid],u1[kid], u2[kid], rorb[kid], b[kid])
ph=np.array(df['phase'])
ax[0].plot(df['phase'],np.convolve(df['flux']-lc,np.ones(20)/20,mode='same'))
ax[1].plot(df['phase'],np.convolve(df['flux_white']-df['model'], np.ones(2)/2, mode='same'))
res=np.convolve(df['flux']-lc,np.ones(20)/20,mode='same')
# popt1, pcov1 = curve_fit(lorz, df['phase'], res,p0=[max(res),min(df['phase'])/2,0.01,max(res),max(df['phase'])/2,0.01], 
#     bounds=([0,min(df['phase']),0.001,0,0,0.001],
#     [max(res), 0, .1,max(res), max(df['phase']),.1]))
peaks, _ = find_peaks(res, height=max(res)/2, distance=10)
ax[0].plot(ph[peaks], res[peaks], "x")

#print(popt1)
print(max(df['phase'])/2)
#ax[0].plot(df['phase'],lorz(df['phase'], *popt1))



Just sorting through the directory

In [ ]:
list1 = np.array(os.listdir('../../processed_directories/go_circles/new_data/'))
list2 = np.array(os.listdir('../../processed_directories/go_circles/new_data_2/'))
for el in list1:
    if(np.any(list2 == el)): 
        #new data storage
        store1 = pd.HDFStore('../../processed_directories/go_circles/new_data/'+el)
        store2 = pd.HDFStore('../../processed_directories/go_circles/new_data_2/'+el)
        df1 = store1['data']
        df2 = store2['data']
        md1 = store1.get_storer('data').attrs.metadata
        md2 = store2.get_storer('data').attrs.metadata
        store1.close()
        store2.close()

        df=pd.DataFrame(list(zip(df2['phase'], df2['flux'], df2['flux_white'], df2['model'], df2['res'], df1['par'], df1['res'])),
        columns=['phase', 'flux','flux_white', 'model_white','res_white','fit','res_fit'])

        tp=md1['tp']
        td=md2['td']
        locdf=df[(df.phase>-2*td/tp) & (df.phase<2*td/tp)]
        out = df[(df.phase<-2*td/tp) | (df.phase>2*td/tp)]

        noise2 = np.std(np.array(out['flux_white']))
        min2 = min(locdf['model_white'])
        noise1 = np.std(np.array(out['flux']))
        min1 = min(locdf['fit'])

        if(-min2/noise2<3 or -min1/noise1<3): continue
        
        new_head = {'ploc_f':md1['ploc'],'pval_f':md1['pval'],'ploc_c':md2['ploc'],'pval_c':md2['pval'],
        'label':md1['label'],'rorb':md1['rorb'],'rpl':md1['rpl'],'u1':md1['u1'],'u2':md1['u2'],'b':md1['b'],
        'tp':md1['tp'],'td':md1['td'], 'snr_f':md1['snr'], 'snr_c':md2['snr'],'sig_snr_c':-min2/noise2, 'sig_snr_f':-min1/noise1}

        #new plotting
        
        fig, ax=plt.subplots(3,2, figsize=(10,10))
        ax[0][0].plot(df['phase'],df['flux_white'], label='flux')
        ax[0][0].plot(df['phase'],df['model_white'], label='model')
        ax[0][0].set_ylabel('flux')
        ax[0][0].set_title('snr:'+str(np.around(-min2/noise2,4)))
        ax[0][1].set_title('snr:'+str(np.around(-min1/noise1,4)))

        ax[0][1].plot(df['phase'],df['flux'], label='flux')
        ax[0][1].plot(df['phase'],df['fit'], label='model')

        ax[1][0].plot(locdf['phase'],locdf['flux_white'], label='flux')
        ax[1][0].plot(locdf['phase'],locdf['model_white'], label='model')
        ax[1][0].set_ylabel('flux')

        ax[1][1].plot(locdf['phase'],locdf['flux'], label='flux')
        ax[1][1].plot(locdf['phase'],locdf['fit'], label='model')

        ax[2][0].plot(locdf['phase'],locdf['res_white'], label='residual')
        ax[2][1].plot(locdf['phase'],locdf['res_fit'], label='residual')
        ax[2][0].scatter(md2['ploc'], md2['pval'])
        ax[2][1].scatter(md1['ploc'], md1['pval'])

        plt.suptitle(el+": "+str(md1['label'])+'\n tp: '+str(md1['tp'])+', td: '+str(md1['td']))
        ax[2][0].set_title('snr:'+str(np.around(md2['snr'],4)))
        ax[2][1].set_title('snr:'+str(np.around(md1['snr'],4)))
        ax[2][0].set_xlabel('phase')
        ax[2][1].set_xlabel('phase')
        ax[2][0].set_ylabel('flux - model')
        ax[0][0].legend()
        ax[1][0].legend()
        ax[2][0].legend()
        ax[0][1].legend()
        ax[1][1].legend()
        ax[2][1].legend()

        plt.savefig('../../processed_directories/go_circles/new_final_fits/'+el+'.jpg')
        store = pd.HDFStore('../../processed_directories/go_circles/new_final_data/'+el)
        store.put('data', df)
        store.get_storer('data').attrs.metadata = new_head
        store.close()
        plt.close()
        
        

In [ ]:
path='C:\\Users\\Hp\Desktop\\temp_finallist\\manda\\'
path='C:\\Users\\Hp\Documents\\FYProj\processed_directories\go_circles\\new_final_fits\\'
entry = os.listdir(path)

for el in entry:
    kid=np.where(np.array(rv_kepid)==str(el[-15:-6]))[0]
    if(len(kid)==0): 
        print("not in catalog",el[-15:-6])
        continue
    kidf=[x for x in kid if(str(rv_pl[x])==str(el[-5:-4]))]
    if(len(kidf)==0): 
        print("not in catalog",el[-5:-4])
        continue
    if(rv_label[kidf[0]]=='CAN'):  
        print(kidf,rv_label[kidf[0]],el) 

SNR Table 

In [ ]:
list = os.listdir('../../processed_directories/go_circles/new_final_data/')
t_el=[]
t_td=[]
t_tp=[]
t_label=[]
t_snr=[]
t_sigsnr=[]
t_rpl=[]
t_rorb=[]
t_u1=[]
t_u2=[]
t_b=[]
for el in list:
    store1 = pd.HDFStore('../../processed_directories/go_circles/new_final_data/'+el)
    df1 = store1['data']
    md1 = store1.get_storer('data').attrs.metadata
    store1.close()

    t_el.append(el)
    t_td.append(md1['td'])
    t_tp.append(md1['tp'])
    t_label.append(md1['label'][0])
    t_snr.append(md1['snr_f'])
    t_sigsnr.append(md1['sig_snr_f'])
    t_rpl.append(md1['rpl'][0])
    t_rorb.append(md1['rorb'][0])
    t_u1.append(md1['u1'][0])
    t_u2.append(md1['u2'][0])
    t_b.append(md1['b'][0])

t_el=np.array(t_el)
t_b=np.array(t_b)
t_label=np.array(t_label)
t_rorb=np.array(t_rorb)
t_td=np.array(t_td)
t_tp=np.array(t_tp)
t_sigsnr=np.array(t_sigsnr)
t_snr=np.array(t_snr)
t_u1=np.array(t_u1)
t_u2=np.array(t_u2)
t_rpl=np.array(t_rpl)
df=pd.DataFrame(zip(t_el, t_td, t_tp, t_label,t_snr, t_sigsnr, t_rpl, t_rorb, t_u1, t_u2, t_b),
    columns=['KID','transit duration', 'transit period', 'label', 'SNR of residual', 'SNR of signal','Rpl/Rst','Rorb/Rst','u1','u2','b'])
df=df.sort_values('SNR of residual',axis=0,ascending=False)
#df.to_csv('../../processed_directories/go_circles/Candidate_List.csv',sep=',', index=False)

for x,i in zip(df['KID'],np.arange(0,len(df))):
    os.rename('../../processed_directories/go_circles/new_final_fits/'+x+'.jpg',
        '../../processed_directories/go_circles/new_final_fits/rank'+str(i)+'_'+x+'.jpg')
    


### Fitting Model (Mandel and Algol 2002)

Run through directory to fit model to the lightcurves: The ones that dont fit properly are in the problist section, may have to approach them manually. 

In [ ]:
#entries = os.listdir('../../processed_directories/go_circles/find_circles_rel/')
entries = np.loadtxt('../../processed_directories/go_circles/problem_nonfits', delimiter=' ', dtype='str')
print(len(entries))
#np.random.shuffle(entries)
problist=[]
for el in entries:
    
    df = pd.read_csv('../../processed_directories/go_circles/find_circles_rel/'+el+'.csv')
    flux = np.array([ x for x in df['flux_l'] if(not(np.isnan(x)))])
    fluxg = np.array([ x for x in df['flux_g'] if(not(np.isnan(x)))])
    phase = np.array(df['phase_l'])[:len(flux)]
    phaseg= np.array(df['phase_g'])[:len(fluxg)]
    #print(min(phaseg), max(phaseg))
   
    try: popt1,pcov1=curve_fit(new_plar_free2, phase, flux, bounds=([0.0001,-np.inf,-np.inf,1], [1,np.inf,np.inf,np.inf]))
    except: 
        print("no fit")
        problist.append(el[:11])
        continue
  
    
    print(el[:9],":",np.round(np.trace(pcov1),3),'ptd: ',np.round(popt1[0],5), 
        np.round((popt1[1]),3), np.round((popt1[2]),3), np.round(popt1[3],3))
     
    # if(popt1[1]>10 or popt1[2]>10):
    #     problist.append(el[:11])
    #     print('overboard')
    #     continue
    # if(popt1[3]<15.0001):
    #     problist.append(el[:11])
    #     print('under rad')
    #     continue
    # if(np.trace(pcov1)>1000):
    #     print("fit variance")
    #     problist.append(el[:11])
    #     continue
    dfg = pd.DataFrame(zip(phaseg,fluxg, new_plar_free2(np.array(phaseg), *popt1)),
        columns=['phase_g','flux_g','model_g'])
    dfl= pd.DataFrame(zip(phase, flux,new_plar_free2(np.array(phase), *popt1)),
        columns=[ 'phase_l', 'flux_l','model_l'])
    df1 = [dfl, dfg]
    df1 = pd.concat(df1, axis=1)

    store = pd.HDFStore('../../processed_directories/go_circles/temp/'+el[:11])
    store.put('data', df1)
    store.get_storer('data').attrs.metadata = {'attr':popt1, 
        'var':np.trace(pcov1)}
    store.close()

np.savetxt('../../processed_directories/go_circles/list_of_nonfits',problist,delimiter=' ',fmt="%s")

In [ ]:
entries = os.listdir('../../processed_directories/go_circles/fit_circles_rel/')
el = entries[100]
store = pd.HDFStore('../../processed_directories/go_circles/fit_circles_rel/'+el)
df = store['data']
metadata = store.get_storer('data').attrs.metadata
store.close()

loc=np.where(np.asarray(rv_kepid)==el[:9])[0]
rv_loc_f = [x for x in loc if(str(rv_pl[x])==el[10:11])]
if(len(rv_loc_f)==0): 
    print("not in catalog")


print(rv_loc_f, av_time[rv_loc_f[0]]/(24*av_p[rv_loc_f[0]]), metadata['var'])
phase = np.array([ x for x in df['phase_l'] if(not(np.isnan(x)))])
flux = np.array([ x for x in df['flux_l'] if(not(np.isnan(x)))])
model = np.array([ x for x in df['model_l'] if(not(np.isnan(x)))])
res = np.convolve(df['flux_l']-df['model_l'], np.ones(10)/10, mode='same')
res = np.array([(el-np.median(res))/(max(res)-min(res)) for el in res])
popt1, pcov1 = curve_fit(gausses, phase, res, bounds=([max(res)/4,0.9*min(phase),0.0001,max(res)/4,0,0.0001], [max(res), 0, .1,max(res), 0.9*max(phase),.1]))
    
popt2, pcov2 = curve_fit(lorz, phase, res, bounds=([max(res)/4,0.9*min(phase),0.0001,max(res)/4,0,00.0001], [max(res), 0, .1,max(res), 0.9*max(phase), .1]))
print(min(phase),max(phase))
print(popt1, popt2)
fig, ax=plt.subplots(2,1)
ax[0].plot(df['phase_l'], df['flux_l'])
ax[0].plot(df['phase_l'], df['model_l'])
ax[1].plot(df['phase_l'], res )
ax[1].plot(phase,gausses(phase, *popt1))
#ax[1].plot(phase, lorz(phase, *popt2))
#ax[1].plot(phase, gausses(phase,0.3,-0.03,0.004,0.3,0.03,0.004))
#ax[0].set_xlim(-0.1,0.1)
#ax[1].set_xlim(-0.1,0.1)


### Finding Residuals
Fitting Gaussian/ Lorentzian Profiles to circles to see where we can get some recognisable peaks...

In [ ]:
entries = os.listdir('../../processed_directories/go_circles/fit_circles_rel/')
problist=[]
for el in entries:
    store = pd.HDFStore('../../processed_directories/go_circles/fit_circles_rel/'+el)
    df = store['data']
    metadata = store.get_storer('data').attrs.metadata
    store.close()

    try: 
        loc=np.where(np.asarray(rv_kepid)==el[:9])[0]
        rv_loc_f = [x for x in loc if(str(rv_pl[x])==el[10:11])]
        #print(rv_loc_f, el)
    except: 
        print("not in catalog")
        continue

    if(len(rv_loc_f)==0): 
        print("not in catalog")
        continue
   
    k = np.ones(10)/10
    flux = np.convolve(np.array([ x for x in df['flux_l'] if(not(np.isnan(x)))]),k,mode='same')
    model = np.convolve(np.array([ x for x in df['model_l'] if(not(np.isnan(x)))]),k,mode='same')
    #flux = np.array(df['flux_l'])
    #model = np.array(df['model_l'])

    res = np.convolve(flux - model, k ,mode='same')
    phase = np.array(df['phase_l'])[:len(res)]
    try: popt1, pcov1 = curve_fit(gausses, phase, res, bounds=([max(res)/4,0.9*min(phase),0.0001,max(res)/4,0,0.0001], [max(res), 0, .1,max(res), 0.9*max(phase),.1]))
    except: 
        print("no gauss fit")
        continue
    
    print(el[:9],'ptd: ', np.abs(popt1[1]-popt1[4]))
     
    dfl = pd.DataFrame(zip(phase,flux,model,res, gausses(phase, *popt1)),
        columns=['phase_l', 'flux_l', 'model_l','residue', 'gaussian'])
    dfg = pd.DataFrame(zip(df['phase_g'],df['flux_g'],df['model_g']),
        columns=['phase_g','flux_g','model_g'])
    df1 = [dfl, dfg]
    df1 = pd.concat(df1, axis=1)
    store = pd.HDFStore('../../processed_directories/go_circles/fit_circles_with_res/'+el[:11])
    store.put('data', df1)
    store.get_storer('data').attrs.metadata = {'label':rv_label[rv_loc_f[0]],'gauss':popt1,'model':metadata['attr'],
        'mod_var':metadata['var'],'gdur':np.abs(popt1[1]-popt1[4]) ,'g_cov':np.trace(pcov1)}
    store.close()


np.savetxt('../../processed_directories/go_circles/list_of_nonfits_residue',problist,delimiter=' ',fmt="%s")

### Cumulative Batch Plotting

In [ ]:
entries = os.listdir('../../processed_directories/go_circles/fit_circles_with_res/')
#np.loadtxt('../../processed_directories/go_circles/hot_jupiters')
np.random.shuffle(entries)
fig, axs = plt.subplots(4,4,figsize=(10,10))


for el,ax in zip(entries[:16],axs.ravel()):
    store = pd.HDFStore('../../processed_directories/go_circles/fit_circles_with_res/'+el)
    df = store['data']
    metadata = store.get_storer('data').attrs.metadata
    #df = pd.read_csv('../../processed_directories/go_circles/find_circles_rel/'+el)
    ax.plot(df['phase_g'],df['flux_g'])
    ax.plot(df['phase_g'],df['model_g'])
    #ax.legend()
    #print(metadata['gauss'], metadata['lorz'])
    #ax.plot(df['phase_l'],df['residue'])
    #ax.plot(df['phase_l'],df['gaussian'])
    #ax.plot(df['phase_l'],df['lorentzian'])
    #ax.set_xlim(-0.2,0.2)
    store.close()

### Selection of promising results

Filter out the ones where the gaussian peaks can actually correspond to the residual features we're looking for

In [ ]:
entries = os.listdir('../../processed_directories/go_circles/fit_circles_with_res/')
for el in entries:
    store = pd.HDFStore('../../processed_directories/go_circles/fit_circles_with_res/'+el)
    df = store['data']
    try: metadata = store.get_storer('data').attrs.metadata
    except: 
        store.close()
        continue
    
    dur1 = metadata['gdur']
    arr1 = metadata['gauss']
    phase = np.array([ x for x in df['phase_l'] if(not(np.isnan(x)))])
    dur = max(phase)-min(phase)
    if((dur1 < dur/2) and arr1[1]>-1.1*dur/4 and arr1[4]<1.1*dur/4): 
            store2 = pd.HDFStore('../../processed_directories/go_circles/analyse_circles_rel/'+el[:11])
        
            data = store['data']
            print(dur/2,dur1,arr1[1], arr1[4])

            store2.put('data', data)
            store2.get_storer('data').attrs.metadata = metadata
            store2.close()
        
    store.close()

### Final Plotting
Plot out the results and manually filter the plots.

In [ ]:
entries = os.listdir('../../processed_directories/go_circles/analyse_circles_rel/')
#np.random.shuffle(entries)

for el in entries[:60]:
    fig, ax = plt.subplots(3,1,figsize=(6,6))
    store = pd.HDFStore('../../processed_directories/go_circles/analyse_circles_rel/'+el)
    df = store['data']
    
    phase=df['phase_l']
    mn = np.mean(df['residue']**2)
    #k = np.ones(2)/2
    ax[0].plot(df['phase_g'],df['flux_g'], label='flux')
    ax[0].plot(df['phase_g'],df['model_g'], label='model')
    ax[1].plot(phase,df['flux_l'], label='flux')
    ax[1].plot(phase,df['model_l'], label='model')
    ax[2].plot(phase,df['residue'], label='residue')
    ax[2].plot(phase,df['gaussian'], label='Gaussian fit')
    ax[0].set_ylabel('Flux')
    ax[2].set_xlabel('Phase')
    ax[2].set_ylabel('Flux - Model')
    ax[2].fill_between(phase, np.sqrt(mn)*np.ones(len(phase)), -np.sqrt(mn)*np.ones(len(phase)), alpha=0.2)
    ax[1].legend()
    ax[0].legend()
    ax[2].legend()
    ax[0].set_title(el[:9]+'\nPl:'+el[10:])
    plt.savefig('../../processed_directories/go_circles/'+el+'.jpg')
    plt.close()
    store.close()

### Plotting for good fits
Make plots for good fits!!!


In [ ]:
entries = os.listdir('../../processed_directories/go_circles/temp/')
#np.random.shuffle(entries)
global i
problist=[]
i=0
for el in entries:
    fig, ax = plt.subplots(3,1,figsize=(8,8))
    store = pd.HDFStore('../../processed_directories/go_circles/temp/'+el)
    df = store['data']
    metadata = store.get_storer('data').attrs.metadata
    attr = metadata['attr']
    if(metadata['var']<1000): 
        i+=1
        print(metadata['var'],attr[1], attr[2])

    # else: 
    #     plt.close()
    #     continue
    phase=df['phase_l']

    # if(attr[1]<-0.99999 or attr[1]>0.99999 or attr[2]<0.0000001 or attr[2]>0.99999):
    #     print('skip')
    #     #plt.close()
    #     #continue
    # else: continue


    problist.append(el)
    props = dict(boxstyle='round', facecolor='black', alpha=0.8, pad=1)
    txt = "$R_{pl}$/$R_{st}$:"+str(np.round(attr[0],4))+"\nu1:"+str(np.round((attr[2]+attr[1])/2,4
        ))+"\nu2:"+str(np.round((attr[2]-attr[1])/2,4))+"\n$R_{orb}/R_{st}$:"+str(np.round(attr[3],4))
    
    txt = "$R_{pl}$/$R_{st}$:"+str(np.round(attr[0],4))+"\nu1:"+str(np.round(attr[1],4
        ))+"\nu2:"+str(np.round(attr[2],4))+"\n$R_{orb}/R_{st}$:"+str(np.round(attr[3],4))


    ax[0].text(0.80, 0.5, txt, fontsize=11,transform=ax[0].transAxes,  horizontalalignment='center',
            verticalalignment='center', linespacing=2, bbox=props, color='white')
    
    mn=  np.mean(np.array([x for x in (df['flux_l']-df['model_l'])**2 if not np.isnan(x)]))
    #k = np.ones(2)/2
    ax[0].plot(df['phase_g'],df['flux_g'], label='flux',color='#b19cd9')
    #ax[0].plot(df['phase_g'],df['model_g'], label='model', color='#311432')
    ax[1].plot(phase,df['flux_l'], label='flux', color='#7a4988', marker='.', ls='None')
    ax[1].plot(phase,df['model_l'], label='model',color='#311432')
    ax[2].plot(phase,df['flux_l']-df['model_l'], label='var:'+str(np.round(metadata['var'],5)), color='#7a4988', marker='.')
    #ax[2].plot(phase,df['gaussian'], label='Gaussian fit')
    ax[0].set_ylabel('Flux')
    ax[2].set_xlabel('Phase')
    ax[2].set_ylabel('Flux - Model')
    ax[2].fill_between(phase, np.sqrt(mn)*np.ones(len(phase)), -np.sqrt(mn)*np.ones(len(phase)), alpha=0.6, 
        color='#b19cd9',label='1$\sigma$')
    ax[1].legend()
    ax[0].legend()
    ax[2].legend()
    ax[0].set_title(el[:9]+'\nPl:'+el[10:])
    plt.savefig('../../processed_directories/go_circles/'+el+'.jpg')
    plt.close()
    store.close()

#np.savetxt('../../processed_directories/go_circles/hot_jupiters',problist,delimiter=' ',fmt="%s")

### Manual Fitting

Here we write a quick handy few lines of code to do stuff manually... to check, and to finish fitting the rest.

In [ ]:
PATH = '../../processed_directories/go_circles/find_circles_rel/'
el = '004349452_1.csv'

df = pd.read_csv(PATH+el)

flux = np.array([ x for x in df['flux_l'] if(not(np.isnan(x)))])
phase = np.array([ x for x in df['phase_l'] if(not(np.isnan(x)))])

#some brute force shit
# uarr=np.linspace(0.1,2, 10)
# varr=np.linspace(0.1,0.9, 10)
# minpopt=[]
# mincov = 1000
# for u in uarr:
#     popt1,pcov1=curve_fit(new_plar_free2, phase, flux, bounds=([0,u-0.01*u,-0.1,10], [1,u+0.01*u,0.9,200]))
#     rchi = np.sum((flux/np.abs(min(flux)) - new_plar_free2(phase, *popt1)/np.abs(min(flux)))**2) / len(flux)
#     print(rchi)
#     if(rchi<mincov):
#         mincov = rchi
#         minpopt = popt1
minpopt,mincov=curve_fit(new_plar, phase, flux, bounds=([0,-0.1,0,10], [1,1,1,200]))


rchi = np.sum((flux/np.abs(min(flux)) - new_plar(phase, *minpopt)/np.abs(min(flux)))**2) / len(flux)
print(rchi)

plt.plot(phase, flux)
#print("fin:",np.trace(mincov))
plt.plot(phase, new_plar(phase, *minpopt))
plt.plot(phase, new_plar(phase, 0.031, 0.02,0.61, 70.9))
print(["{:.5f}".format(x) for x in minpopt])
plt.show()

Quick check on theoretical transit sims


In [ ]:
df = pd.read_csv('2d3d_0.2R_limb_circ_corr.csv', delimiter=',')
np.random.seed(10000)
#noise = np.random.normal(1,0.002777,len(df))
noise = np.ones(len(df))

d2=df['2d']-1
d3=df['3d']-1
ph = np.linspace(-1/5, 1/5, len(df))/2
d2n = np.convolve((df['2d']*noise-1),np.ones(5)/5, mode='same')
d3n = (df['3d']*noise-1)
#std2 = df['2dstd']
#std3 = df['3dstd']
#print(np.mean(np.array([d2/el for el in std2 if(el>0)])))
#print(np.mean(np.array([d3/el for el in std3 if(el>0)])))
#print(np.mean(std2), np.mean(std3))

par2d, cov2d=curve_fit(new_plar_v2, ph, d2, bounds=([0,0,0,2,0], [0.5,1,1,7,1]))
par3d, cov3d=curve_fit(new_plar_v2, ph, d3, bounds=([0,0,0,2,0], [0.5,1,1,7,1]))
#print(["{:.5f}".format(x) for x in minpopt])

fig, ax = plt.subplots(2,2, figsize=(10,7))
plt.suptitle('Sim vs Model\n Rpl = 0.1 Rst, Rorb = 2 Rst, u = 0.0\n $\sigma_mc = 10^{-3}$')

props = dict(boxstyle='round', facecolor='black', alpha=0.8, pad=1)
txt = "$R_{pl}$/$R_{st}$:"+str(np.round(par2d[0],4))+"\nu1:"+str(np.round(par2d[1],4
        ))+"\nu2:"+str(np.round(par2d[2],4))+"\n$R_{orb}/R_{st}$:"+str(np.round(par2d[3],4))

props2 = dict(boxstyle='round', facecolor='black', alpha=0.8, pad=1)
txt2 = "$R_{pl}$/$R_{st}$:"+str(np.round(par3d[0],4))+"\nu1:"+str(np.round(par3d[1],4
        ))+"\nu2:"+str(np.round(par3d[2],4))+"\n$R_{orb}/R_{st}$:"+str(np.round(par3d[3],4))


ax[0][1].text(0.75, 0.75, txt, fontsize=11,transform=ax[0][1].transAxes,  horizontalalignment='center',
            verticalalignment='center', linespacing=1, bbox=props, color='white')

ax[0][0].text(0.75, 0.75, txt2, fontsize=11,transform=ax[0][0].transAxes,  horizontalalignment='center',
            verticalalignment='center', linespacing=1, bbox=props2, color='white')    

ax[0][0].plot(ph, d3n, label='sim', zorder=1)
ax[0][1].plot(ph, d2n, label='sim', zorder=1)
#ax[0][0].errorbar(ph, d3, std3,None, label='sim', zorder=2)
#ax[0][1].errorbar(ph, d2, std2,None, label='sim', zorder=2)

ax[0][0].plot(ph, new_plar_v2(ph, *par3d), label='model')
ax[0][1].plot(ph, new_plar_v2(ph, *par2d), label='model')
# ax[0][0].plot(ph, new_plar_free2(ph, 0.01, 0.45, 0.0, 2), label='model')
# ax[0][1].plot(ph, new_plar_free2(ph, 0.01, 0.45, 0.0, 2), label='model')
ax[0][0].legend(loc='lower left')
ax[1][1].set_xlabel('Phase')
ax[1][0].set_xlabel('Phase')
ax[1][0].set_ylabel('sim - model')
ax[0][0].set_ylabel('flux')
ax[0][1].legend(loc='lower left')
ax[1][0].set_title('Residuals')
ax[1][1].set_title('Residuals')
ax[1][0].plot(ph, d3-new_plar_v2(ph, *par3d))
ax[1][1].plot(ph, d2-new_plar_v2(ph, *par2d))
# ax[1][1].set_ylim(-0.00013,0.00013)
# ax[1][0].set_ylim(-0.00013,0.00013)
#plt.savefig('th_sim_fit_0.1R_n2.jpg')
print(par3d, par2d)
# dfop = pd.DataFrame(zip(ph,d3n,d2n,new_plar_free2(ph, *par3d),new_plar_free2(ph, *par2d)), columns=['phase', 'flux','flux2d', 'model','model2d'])
# dfop.to_csv('fprez_simp.csv', sep=',',index=False)
# plt.savefig('fprez_fit.jpg')



Signal to Noise Ratio Analysis

In [ ]:
#d2 d3 std2 std3
df = np.loadtxt('2d3d_0.2R_circ.csv', delimiter=',')
np.random.seed(10000)

std_n = np.logspace(-1, -5, 20)
d2=df[:,1]-1
d3=df[:,3]-1
ph = df[:,0]*0.5/ np.pi
d2n = (df[:,1]*noise-1)
d3n = (df[:,3]*noise-1)
std2 = df[:,2]
std3 = df[:,4]

parrarr2d=[]
parrarr3d=[]
snrarr2=[]
snrarr3=[]
for el in std_n:
    noise = np.random.normal(1,el,len(df))
    d2n = (df[:,1]*noise-1)
    d3n = (df[:,3]*noise-1)
    #snr2 = np.mean(d2/(d2-d2n)**2)
    #snr3 = np.mean(d3/(d3-d3n)**2)
    #snrarr2.append(snr2)
    #snrarr3.append(snr3)
    par2d, cov2d=curve_fit(new_plar_free2, ph, d2n, bounds=([0,0,0,1], [1,1,1,10]))
    par3d, cov3d=curve_fit(new_plar_free2, ph, d3n, bounds=([0,0,0,1], [1,1,1,10]))
    parrarr2d.append(par2d)
    parrarr3d.append(par3d)

parrarr2d = np.asarray(parrarr2d)
parrarr3d = np.asarray(parrarr3d)
fig, ax = plt.subplots(2,2,figsize=(10,10))
plt.suptitle('Parameter Evolution\nRpl/Rst:0.2, u1:0.6, u2:0.0, Rorb/Rst:6\n \n $\sigma_{mc} = 10^{-3}$')
ax[0][0].plot(std_n,parrarr2d[:,0], marker='.', label='2d')
ax[0][0].plot(std_n,parrarr3d[:,0], marker='.', label='3d')
ax[0][0].set_title('Rpl/Rst')
ax[0][0].set_xscale('log')
ax[0][0].legend()
ax[0][1].plot(std_n,parrarr2d[:,1], marker='.')
ax[0][1].plot(std_n,parrarr3d[:,1], marker='.')
ax[0][1].set_title('u1')
ax[0][1].set_xscale('log')
ax[1][0].plot(std_n,parrarr2d[:,2], marker='.')
ax[1][0].plot(std_n,parrarr3d[:,2], marker='.')
ax[1][0].set_title('u2')
ax[1][0].set_xscale('log')
ax[1][0].set_xlabel('std of noise')
ax[1][1].plot(std_n,parrarr2d[:,3], marker='.')
ax[1][1].plot(std_n,parrarr3d[:,3], marker='.')
ax[1][1].set_title('Rorb/Rst')
ax[1][1].set_xscale('log')
ax[1][1].set_xlabel('std of noise')
plt.savefig('snr_calc.jpg')
plt.show()



Some labour work...

In [ ]:
fins = os.listdir('../../processed_directories/go_circles/plots_rel/')
takeout1 = os.listdir('../../processed_directories/go_circles/fit_circles_rel/')
takeout2 = os.listdir('../../processed_directories/go_circles/temp/')

for el in fins:
    loc = np.where(np.asarray(takeout2)==el[:11])[0]
    #print(loc)

    if(len(loc)>0):
        store1 = pd.HDFStore('../../processed_directories/go_circles/temp/'+el[:11])
        try: df1 = store1['data']
        except: continue
        metadata = store1.get_storer('data').attrs.metadata
        metadata['method']='ind'
        print(metadata)

        store2 = pd.HDFStore('../../processed_directories/go_circles/final_fits/'+el[:11])
        
        data = store1['data']

        store2.put('data', data)
        store2.get_storer('data').attrs.metadata = metadata
        store2.close()
        store1.close()
    
    

In [ ]:
takeout = os.listdir('../../processed_directories/go_circles/find_circles_rel/')
takin = os.listdir('../../processed_directories/go_circles/final_fits/')
problist=[]

for el in takeout:
    if(np.any(np.array(takin)==el[:11])):
        continue
    else: problist.append(el[:11])

np.savetxt('../../processed_directories/go_circles/problem_nonfits',problist,delimiter=' ',fmt="%s")
    


### Optimal Kepler Parameters
Plotting histograms of optimal kepler parameters to see what values we should take to construct our ideal simulation

In [ ]:
from astropy.io import ascii
df = ascii.read('../../Catalogs/ex_TCE_extra.tab')
# print(df['koi_ldm_coeff1'], df['koi_ldm_coeff2'], df['koi_dor'], df['koi_ror'])
rorb = np.array([ x for x in df['koi_dor'] if not np.isnan(x)])
rpl = np.array([ x for x in df['koi_ror'] if not np.isnan(x)])
#print(np.sum(rorb>1000))
rpl = np.array([x for x in rpl if x<1])
rorb = np.array([x for x in rorb if x>1 and x<500])
print(rorb)

fig, ax = plt.subplots(2,1,figsize=(6,8))
yo, xo, _ = ax[0].hist(rorb, bins=200)
yp, xp, _ = ax[1].hist(rpl, bins=100)

vp=(xp[np.where(yp == yp.max())[0]]+xp[np.where(yp == yp.max())[0]+1])[0]/2
vo=(xo[np.where(yo == yo.max())[0]]+xo[np.where(yo == yo.max())[0]+1])[0]/2

print(vp,vo)

ax[1].set_title('$R_{pl}/R_{st}$\n Peak:'+str(np.round(vp,4)))
ax[0].set_title('$R_{orb}/R_{st}$\n Peak:'+str(np.round(vo,4)))
ax[1].set_xlim(-0.2,1)

plt.savefig('parameter_histogram2.jpg')


In [ ]:
us = np.array([[x,y] for x,y in zip(df['koi_ldm_coeff1'], df['koi_ldm_coeff2'])])
us=np.array([el for el in us if not np.any(np.isnan(el))])
print(np.sum(np.isnan(us)))
z,x,y,_ = plt.hist2d(us[:,0], us[:,1], bins=(30,30))
#print(el)
#print(z)
wh = np.where(z==max(z.reshape(-1)))
print(wh)
print(max(yo),[x[wh[0]],y[wh[1]]], [x[wh[0]+1], y[wh[1]+1]])
v1 = (x[wh[0]][0]+x[wh[0]+1][0])/2
v2 = (y[wh[1]][0]+y[wh[1]+1][0])/2
plt.xlabel('u1')
plt.title('Limb Coefficients\nPeak> u1: '+str(np.round(v1,4))+', u2: '+str(np.round(v2,4)))
plt.ylabel('u2')

plt.savefig('parameter_histogram.jpg')
plt.show()

In [ ]:
snr = np.array([ x for x in df['koi_model_snr'] if not np.isnan(x)])
print(len(snr))
print(np.sum(snr>5000))
y,x, _ = plt.hist(snr, bins=100)
#print(x,y)
plt.xlim(-100,4000)
plt.title('Histogram SNR')
plt.savefig('snr_hist.jpg')
plt.show()